In [10]:
df = pd.read_csv('C:/Users/oyeye/EVProject/rendata.csv')

In [25]:
# Forecast EV Sales Based on Renewable Energy Growth and Govt. Subsidies
def forecast_ev_sales(df, growth_factor = 1.05):
    df['renewable_energy_capacity_mw_growth'] = df.groupby('state')['renewable_energy_capacity_mw'].pct_change().fillna(0) * growth_factor
    df['government_subsidy_impact'] = df.groupby('state')['government_subsidy_ev'].pct_change().fillna(0)
    df['ev_sales_growth'] = df.groupby('state')['total_ev_sales'].pct_change().fillna(0)
    df['total_ev_sales_previous'] = df['total_ev_sales'].shift(1)
    
    def calc_ev_renewable(row):
        renewable_energy_capacity_mw_growth = row['renewable_energy_capacity_mw_growth']
        government_subsidy_impact = row['government_subsidy_impact']
        ev_sales_growth = row['ev_sales_growth']
        total_ev_sales_current = row['total_ev_sales']
        total_ev_sales_previous = row['total_ev_sales_previous']
        state = row['state']
        benchmark = 0.05
            
        # Benchmark Growth Comparison
        if renewable_energy_capacity_mw_growth < benchmark:
            benchmark_growth = 'Underperforming Market'
            
        elif ev_sales_growth > 0.1:
            benchmark_growth = 'Independent Growth Market'
            
        elif (state == state and total_ev_sales_current < total_ev_sales_previous):
            benchmark_growth = 'Declining EV Market'
        
        else:
            benchmark_growth = 'Stable Market'
        
        # Performance Categories
        if renewable_energy_capacity_mw_growth >= benchmark + 0.05:
            perf_cat = 'Outstanding'
            
        elif benchmark <= renewable_energy_capacity_mw_growth < benchmark + 0.05:
            perf_cat = 'Good'
            
        elif renewable_energy_capacity_mw_growth < benchmark:
            perf_cat = 'Underperforming'
            
        else:
            perf_cat = 'unclassified'
            
        
        # Market Classification
        if (renewable_energy_capacity_mw_growth == 0 and government_subsidy_impact == 0.1):
            market_class = 'Policy Driven Market'
            
        elif (renewable_energy_capacity_mw_growth == 0.1 and government_subsidy_impact == 0.1):
            market_class = 'Booming EV Market'
            
        elif (renewable_energy_capacity_mw_growth < 0 and government_subsidy_impact == 0):
            market_class = 'Market Contraction'
            
        else:
            market_class = 'unclassified'
            
        return pd.Series({'benchmark_growth' : benchmark_growth,
                         'perf_cat' : perf_cat,
                         'market_class' : market_class})
    
    results = df.apply(calc_ev_renewable, axis = 1)
    df = pd.concat([df, results], axis = 1)
    return df

In [31]:
# Forecast Renewable Energy Capacity Growth for Each State
# Forecast the renewable energy capacity growth for each state based on previous years' growth
def forecast_renewable_energy_capacity(df):
    df['ren_energy_growth_rate'] = df.groupby('state')['renewable_energy_capacity_mw'].pct_change().fillna(0)
    df['renewable_energy_capacity_previous'] = df['renewable_energy_capacity_mw'].shift(1)
    
    def calculate_renewable_forecast(row):
        state = row['state']
        year = row['year']
        renewable_energy_capacity_current = row['renewable_energy_capacity_mw']
        renewable_energy_capacity_previous = row['renewable_energy_capacity_previous']
        ren_energy_growth_rate = row['ren_energy_growth_rate']
        growth_factor = 1.05
        
        forecasted_capacity = renewable_energy_capacity_current * (1 + growth_factor)
        
        if ren_energy_growth_rate < 0.05:
            market_class = 'Low Energy Market'
            
        elif renewable_energy_capacity_current < renewable_energy_capacity_previous:
            market_class = 'Declining Energy Market'
            
        elif (renewable_energy_capacity_previous == 0 and renewable_energy_capacity_current > 0):
            market_class = 'Emerging Energy Market'
            
        elif ren_energy_growth_rate > 0.1:
            market_class = 'Booming Market'
            
        else:
            market_class = 'No Market'
            
        return pd.Series({'forecasted_capacity' : forecasted_capacity,
                         'market_class' : market_class})
    
    results = df.apply(calculate_renewable_forecast, axis = 1)
    df = pd.concat([df, results], axis = 1)
    return df

In [41]:
# CO2 Emissions Reduction per EV for Each State
def carbon_emissions_reduction(df):
    df['total_ev_sales_prev'] = df['total_ev_sales'].shift(1)
    
    def calc_emiss_red(row):
        state = row['state']
        total_ev_sales_current = row['total_ev_sales']
        total_ev_sales_prev = row['total_ev_sales_prev']
        co2_emissions_reduction_tons = row['co2_emissions_reduction_tons']
        reduction_per_ev = co2_emissions_reduction_tons / total_ev_sales_current
        
        if reduction_per_ev > 0.75:
            emissions_class = 'High Efficiency'
            
        elif total_ev_sales_current < total_ev_sales_prev:
            emissions_class = 'Declining Efficiency'
            
        else:
            emissions_class = 'Stable Efficiency'
            
        return pd.Series({'reduction_per_ev' : reduction_per_ev,
                         'emissions_class' : emissions_class})
    
    res = df.apply(calc_emiss_red, axis = 1)
    df = pd.concat([res, df], axis = 1)
    return df

In [9]:
# Predict Market Performance Based on Government Subsidy and Energy Capacity Growth
def market_performance_prediction(df):
    df['renewable_energy_growth'] = df['renewable_energy_capacity_mw'].pct_change()
    df['govt_subsidy_growth'] = df['government_subsidy_ev'].pct_change()
    
    def calc_market_predict(row):
        renewable_energy_growth = row['renewable_energy_growth']
        govt_subsidy_growth = row['govt_subsidy_growth']
        
        market_growth_factor = (renewable_energy_growth + govt_subsidy_growth) / 2
        
        if market_growth_factor > 0.1:
            market_perf = 'Booming Market'
            
        elif 0 < market_growth_factor <= 0.1:
            market_perf = 'Stable Market'
            
        else:
            market_perf = 'Declining Market'
            
        return pd.Series({'market_perf' : market_perf,
                         'market_growth_factor' : market_growth_factor})
    
    res = df.apply(calc_market_predict, axis = 1)
    df = pd.concat([df, res], axis = 1)
    return df

In [25]:
# Multi-Year EV Sales Forecast with Dynamic Growth
def multi_year_forecast(df):
    base_growth_rate = 0.05
    years = 5
    forcast_sales = []
    
    for state, group in df.groupby('state'):
        current_total_ev_sales = group['total_ev_sales'].iloc[-1]
        current_renewable_energy_capacity_mw = group['renewable_energy_capacity_mw'].iloc[-1]
        current_government_subsidy_ev = group['government_subsidy_ev'].iloc[-1]
        growth_rate = base_growth_rate
        
        for year in range(1, years + 1):
            if group['renewable_energy_capacity_mw'].pct_change().iloc[-1] > 0.1:
                growth_rate += 0.02
                
            if current_government_subsidy_ev > 5000:
                growth_rate += (current_government_subsidy_ev - 5000) / 10000 * 0.005
                
            if current_total_ev_sales > 100000:
                growth_rate = min(growth_rate, 0.15)
                
            if group['renewable_energy_capacity_mw'].pct_change().iloc[-1] < 0:
                growth_rate = (1 - 0.03) * growth_rate
                
            
            forecast_ev_sales = current_total_ev_sales * (1 + growth_rate)
            
            forcast_sales.append({'state' : state,
                                 'year' : group['year'].max() + year,
                                 'sales' : forecast_ev_sales})
            
            
            current_total_ev_sales = forecast_ev_sales
            
    
    forecast_result = pd.DataFrame(forcast_sales)
    return forecast_result

In [48]:
# Renewable Energy Capacity Forecast with Battery Storage Growth
def renewable_energy_forecast(df):
    base_growth_rate = 0.05
    years = 5
    forecast_results = []
    
    """
    Forecast renewable energy capacity for each state dynamically with battery storage impact.

    Parameters:
    df (DataFrame): Contains 'state', 'renewable_energy_capacity_mw', 'battery_storage_capacity_mwh'.
    years (int): Number of years to forecast. Default is 5.
    base_growth_rate (float): Base growth rate for renewable energy capacity. Default is 5%.

    Returns:
    DataFrame: Includes forecasted renewable energy capacity for each state.
    
    """
        
    for state, group in df.groupby('state'):
        current_renewable_energy_capacity_mw = group['renewable_energy_capacity_mw'].iloc[-1]
        current_battery_storage_capacity_mwh = group['battery_storage_capacity_mwh'].iloc[-1]
        growth_rate = base_growth_rate
        
        for y in range(1, years + 1):
            
            # Batter impact for every 100 MWh increase in storage
            increase = current_battery_storage_capacity_mwh // 100
            if increase > 0:
                growth_rate *= increase * 0.01
                battery_impact = growth_rate
            
            # Battery Boost for every additional 100 MWh of battery capacity
            if increase > 0:
                current_renewable_energy_capacity_mw *= increase * 1.01
                
            # Plateau Effect. Cap growth rate at 10% for states exceeding 1,000 MW capacity
            if current_renewable_energy_capacity_mw > 1000:
                growth_rate = min(growth_rate, 0.1)
            
            # Negative Growth. Reduce growth rate by 2% for the following year for renewable capacity decline
            if group['renewable_energy_capacity_mw'].pct_change().iloc[-1] < 0:
                growth_rate *= (1 - 0.02)
                
            forecasted_renewable_energy_capacity = current_renewable_energy_capacity_mw * (1 + base_growth_rate + battery_impact)
            
            forecast_results.append({'state' : state,
                                     'year' : group['year'].max() + y,
                                    'current_renewable_energy_capacity_mw' : current_renewable_energy_capacity_mw,
                                    'forecasted_renewable_energy_capacity' : forecasted_renewable_energy_capacity})
            
            
            current_renewable_energy_capacity_mw = forecasted_renewable_energy_capacity
            
    forecasted_df = pd.DataFrame(forecast_results)
    return forecasted_df        

In [90]:
# Forecast CO2 Emissions Reduction Based on Renewable Energy and EV Adoption
def co2_emissions_reduction_forecast(df):
    
    """
    Forecast CO2 emissions reduction for each state based on renewable energy growth and EV adoption rates.

    Parameters:
    df (DataFrame): Contains 'state', 'renewable_energy_capacity_mw', 'total_ev_sales', 'co2_emissions_reduction_tons'.
    years (int): Number of years to forecast. Default is 5.
    base_growth_rate (float): Base annual growth rate for CO2 reduction. Default is 5%.

    Returns:
    DataFrame: Includes forecasted CO2 emissions reduction for each state.
    
    """
        
    years = 5
    base_growth_rate = 0.05
    emissions_reduction = []
    
    for state, group in df.groupby('state'):
        current_renewable_energy_capacity_mw = group['renewable_energy_capacity_mw'].iloc[-1]
        current_total_ev_sales =  group['total_ev_sales'].iloc[-1]
        current_co2_emissions_reduction_tons =  group['co2_emissions_reduction_tons'].iloc[-1]
        growth_rate = base_growth_rate
        projected_ev_Sales = 0.05
        
        for y in range(1, years + 1):
            
            # Accelerated Impact of EV Sales
            if current_renewable_energy_capacity_mw == 1.1 * group['renewable_energy_capacity_mw'].iloc[-2]:
                current_co2_emissions_reduction_tons *= 1.05
            
            # Plateau Effect
            if current_renewable_energy_capacity_mw > 1000:
                current_co2_emissions_reduction_tons *= (1 - 0.03)
                
            # Negative Impact
            if current_total_ev_sales < group['total_ev_sales'].iloc[-2]:
                current_co2_emissions_reduction_tons *= (1 - 0.02)
                
            # No Growth
            if current_renewable_energy_capacity_mw == 0:
                current_co2_emissions_reduction_tons = current_co2_emissions_reduction_tons
                
            ev_impact = projected_ev_Sales * 0.05
            
            forecasted_co2_reduction = current_co2_emissions_reduction_tons * (1 + base_growth_rate + ev_impact)
            
            emissions_reduction.append({'state' : state,
                                        'year' : group['year'].max() + y,
                                        'current_co2_emissions_reduction_tons' : current_co2_emissions_reduction_tons,
                                        'forecasted_co2_reduction' : forecasted_co2_reduction})
            
            #forecasted_co2_reduction = current_co2_emissions_reduction_tons
            
    df_forecast = pd.DataFrame(emissions_reduction)
    return df_forecast

In [102]:
def forecasted_grid_efficiency(df, years = 5):
    forecast_results = []
    
    for state, group in df.groupby('state'):
        current_grid_efficiency = group['grid_efficiency_percentage'].iloc[-1]
        current_grid_efficiency = current_grid_efficiency / 100
        current_renewable_capacity = group['renewable_energy_capacity_mw'].iloc[-1]
        current_storage_capacity = group['battery_storage_capacity_mwh'].iloc[-1]
        
        for y in range(1, years + 1):
            renewable_growth = (current_renewable_capacity / 100) * 0.3
            battery_storage_impact = (current_storage_capacity / 100) * 0.2
            
            # Renewable Boost
            if group['renewable_energy_capacity_mw'].pct_change().iloc[-1] > 0.1:
                current_grid_efficiency *= 1.005
            
            # Storage Acceleration
            if group['battery_storage_capacity_mwh'].pct_change().iloc[-1] > 0.15:
                current_grid_efficiency *= 1.003
            
            # Declining Infrastructure
            if current_renewable_capacity < group['renewable_energy_capacity_mw'].iloc[-2]:
                current_grid_efficiency *= 0.995
            
            # Plateau Limit
            current_grid_efficiency = min(current_grid_efficiency, 0.99)
            
            forecasted_grid_efficiency = current_grid_efficiency + (renewable_growth + battery_storage_impact)
            
            forecast_results.append({'state' : state,
                                    'year' : group['year'].max() + y,
                                    'current_grid_efficiency' : current_grid_efficiency,
                                    'forecasted_grid_efficiency' : forecasted_grid_efficiency})
            
    df_forecast = pd.DataFrame(forecast_results)
    return df_forecast